除前面讲的分页类型外（该类型的分页可通过for循环获得url连接，进行信息爬取），还有两种类型的分页方式，用以上方法不行


In [1]:
import requests
from bs4 import BeautifulSoup as bs
header={'User-Agent':'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.70 Safari/537.36'}

# 普通分页使用for循环

In [2]:
urls=['https://tieba.baidu.com/p/2256306796?pn={}'.format(i) for i in range(1,3)]
for url in urls:
    print(url)

https://tieba.baidu.com/p/2256306796?pn=1
https://tieba.baidu.com/p/2256306796?pn=2


# 滚动分页
测试网站--豆瓣电影中的分类排行
https://movie.douban.com/typerank?type_name=%E5%89%A7%E6%83%85&type=11&interval_id=100:90&action=

## 网页原代码说明
打开以上链接，通过网下滑动会不断的出来新的电影信息，但url仍然为最原始的，

1、审查元素，滑动后，在network---name中会新出来一部分信息：包括图片、链接等等……

2、注意如下类似的链接

https://movie.douban.com/j/chart/top_list?type=11&interval_id=100%3A90&action=&start=0&limit=20
https://movie.douban.com/j/chart/top_list?type=11&interval_id=100%3A90&action=&start=20&limit=20
https://movie.douban.com/j/chart/top_list?type=11&interval_id=100%3A90&action=&start=40&limit=20
https://movie.douban.com/j/chart/top_list?type=11&interval_id=100%3A90&action=&start=60&limit=20

3、点击Headers旁边的Preview会观察到里面保存的信息，即为电影详细情况，

第一条链接表示前20条信息，第二个链接存储20-40条信息，依次类推，这些链接类似于普通分页中的url

## 爬前20条信息

### 请求并获得响应

In [3]:
url="https://movie.douban.com/j/chart/top_list?type=11&interval_id=100%3A90&action=&start=0&limit=20"
header={'User-Agent':'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.70 Safari/537.36'}
res=requests.get(url,headers=header)
h=res.text   #用的是text---返回的是字符串格式

In [4]:
h

'[{"rating":["9.7","50"],"rank":1,"cover_url":"https://img3.doubanio.com\\/view\\/photo\\/s_ratio_poster\\/public\\/p480747492.jpg","is_playable":true,"id":"1292052","types":["犯罪","剧情"],"regions":["美国"],"title":"肖申克的救赎","url":"https:\\/\\/movie.douban.com\\/subject\\/1292052\\/","release_date":"1994-09-10","actor_count":25,"vote_count":1817358,"score":"9.7","actors":["蒂姆·罗宾斯","摩根·弗里曼","鲍勃·冈顿","威廉姆·赛德勒","克兰西·布朗","吉尔·贝罗斯","马克·罗斯顿","詹姆斯·惠特摩","杰弗里·德曼","拉里·布兰登伯格","尼尔·吉恩托利","布赖恩·利比","大卫·普罗瓦尔","约瑟夫·劳格诺","祖德·塞克利拉","保罗·麦克兰尼","芮妮·布莱恩","阿方索·弗里曼","V·J·福斯特","弗兰克·梅德拉诺","马克·迈尔斯","尼尔·萨默斯","耐德·巴拉米","布赖恩·戴拉特","唐·麦克马纳斯"],"is_watched":false},{"rating":["9.6","50"],"rank":2,"cover_url":"https://img3.doubanio.com\\/view\\/photo\\/s_ratio_poster\\/public\\/p2561716440.jpg","is_playable":true,"id":"1291546","types":["剧情","爱情","同性"],"regions":["中国大陆","中国香港"],"title":"霸王别姬","url":"https:\\/\\/movie.douban.com\\/subject\\/1291546\\/","release_date":"1993-07-26","actor_count":25,"vote_count":1337325,"score":"9.6"

### 借助json进行解析
将得到的html进行格式解析，方便观察和分析

#### 法一--json在线翻译

#### 法二-json模块
实现Python数据和json之间的一个转换，将字符串类型的通过json转换成字典类型的

返回的是列表格式，列表中包含一个个字典类型的数据[{},{},{},……]

In [4]:
import json
html=json.loads(h)   #返回列表，便于遍历

In [5]:
html

[{'rating': ['9.7', '50'],
  'rank': 1,
  'cover_url': 'https://img3.doubanio.com/view/photo/s_ratio_poster/public/p480747492.jpg',
  'is_playable': True,
  'id': '1292052',
  'types': ['犯罪', '剧情'],
  'regions': ['美国'],
  'title': '肖申克的救赎',
  'url': 'https://movie.douban.com/subject/1292052/',
  'release_date': '1994-09-10',
  'actor_count': 25,
  'vote_count': 1821402,
  'score': '9.7',
  'actors': ['蒂姆·罗宾斯',
   '摩根·弗里曼',
   '鲍勃·冈顿',
   '威廉姆·赛德勒',
   '克兰西·布朗',
   '吉尔·贝罗斯',
   '马克·罗斯顿',
   '詹姆斯·惠特摩',
   '杰弗里·德曼',
   '拉里·布兰登伯格',
   '尼尔·吉恩托利',
   '布赖恩·利比',
   '大卫·普罗瓦尔',
   '约瑟夫·劳格诺',
   '祖德·塞克利拉',
   '保罗·麦克兰尼',
   '芮妮·布莱恩',
   '阿方索·弗里曼',
   'V·J·福斯特',
   '弗兰克·梅德拉诺',
   '马克·迈尔斯',
   '尼尔·萨默斯',
   '耐德·巴拉米',
   '布赖恩·戴拉特',
   '唐·麦克马纳斯'],
  'is_watched': False},
 {'rating': ['9.6', '50'],
  'rank': 2,
  'cover_url': 'https://img3.doubanio.com/view/photo/s_ratio_poster/public/p2561716440.jpg',
  'is_playable': True,
  'id': '1291546',
  'types': ['剧情', '爱情', '同性'],
  'regions': ['中国大陆', '中国香港']

## 多页爬取

In [6]:
import pandas as pd

### 提取所有信息 

In [7]:
data=[]
for i in range(3):
    url="https://movie.douban.com/j/chart/top_list?type=11&interval_id=100%3A90&action=&start={}&limit=20".format(i*20)
    res=requests.get(url,headers=header).text
    html=json.loads(res)
    data+=html

### 转化成DataFrame类型

In [8]:
df=pd.DataFrame(data)
df.head(1)

,actor_count,actors,cover_url,id,is_playable,is_watched,rank,rating,regions,release_date,score,title,types,url,vote_count
0,25,"[蒂姆·罗宾斯, 摩根·弗里曼, 鲍勃·冈顿, 威廉姆·赛德勒, 克兰西·布朗, 吉尔·贝罗...",https://img3.doubanio.com/view/photo/s_ratio_p...,1292052,True,False,1,"[9.7, 50]",[美国],1994-09-10,9.7,肖申克的救赎,"[犯罪, 剧情]",https://movie.douban.com/subject/1292052/,1821406


### 数据库存储

In [10]:
from pymongo import MongoClient
con=MongoClient('localhost',27017)
db=con.stats
collection=db.movies
collection.insert_many(data)

### 筛选所需内容

In [34]:
df1=df.loc[:,['title','rank','score','types','regions','release_date','url']]
df1.head(3)

,title,rank,score,types,regions,release_date,url
0,肖申克的救赎,1,9.7,"[犯罪, 剧情]",[美国],1994-09-10,https://movie.douban.com/subject/1292052/
1,霸王别姬,2,9.6,"[剧情, 爱情, 同性]","[中国大陆, 中国香港]",1993-07-26,https://movie.douban.com/subject/1291546/
2,控方证人,3,9.6,"[剧情, 犯罪, 悬疑]",[美国],1957-12-17,https://movie.douban.com/subject/1296141/


# 点击“加载更多”
该类型的分页，点击前、后如下。貌似和之前的一样，但是将任意一个url在浏览器中打开，会$$回到最初的start=0的界面，不会显示加载更多后的内容。$$

https://movie.douban.com/explore#!type=movie&tag=%E7%83%AD%E9%97%A8&sort=recommend&page_limit=20&page_start=0

https://movie.douban.com/explore#!type=movie&tag=%E7%83%AD%E9%97%A8&sort=recommend&page_limit=20&page_start=20

https://movie.douban.com/explore#!type=movie&tag=%E7%83%AD%E9%97%A8&sort=recommend&page_limit=20&page_start=40

## 网页解说
类似于前面的滚动分页，也可以找到类似的存储信息的

https://movie.douban.com/j/search_subjects?type=movie&tag=%E7%83%AD%E9%97%A8&sort=recommend&page_limit=20&page_start=0

https://movie.douban.com/j/search_subjects?type=movie&tag=%E7%83%AD%E9%97%A8&sort=recommend&page_limit=20&page_start=20

In [36]:
import pandas as pd
import json

In [42]:
url="https://movie.douban.com/j/search_subjects?type=movie&tag=%E7%83%AD%E9%97%A8&sort=recommend&page_limit=20&page_start=0"
res=requests.get(url,headers=header).text

## json解析
注意----返回的结果是字符串类型，且结构和滚动分页中的不同----'{'subjects'：[{},{},{},……]}'

只有[{},{},{},……]中的内容才是所需，通过json先将其转换成{'subjects'：[{},{},{},……]}，再提取出来[{},{},{},……]，再进行遍历

In [46]:
res  

'{"subjects":[{"rate":"5.9","cover_x":1080,"title":"囧妈","url":"https:\\/\\/movie.douban.com\\/subject\\/30306570\\/","playable":true,"cover":"https://img3.doubanio.com\\/view\\/photo\\/s_ratio_poster\\/public\\/p2581835383.jpg","id":"30306570","cover_y":1542,"is_new":false},{"rate":"8.5","cover_x":3158,"title":"1917","url":"https:\\/\\/movie.douban.com\\/subject\\/30252495\\/","playable":false,"cover":"https://img1.doubanio.com\\/view\\/photo\\/s_ratio_poster\\/public\\/p2570243317.jpg","id":"30252495","cover_y":5000,"is_new":false},{"rate":"6.1","cover_x":495,"title":"丑女之战","url":"https:\\/\\/movie.douban.com\\/subject\\/30162754\\/","playable":false,"cover":"https://img3.doubanio.com\\/view\\/photo\\/s_ratio_poster\\/public\\/p2518187010.jpg","id":"30162754","cover_y":650,"is_new":true},{"rate":"6.1","cover_x":2364,"title":"法医宋慈","url":"https:\\/\\/movie.douban.com\\/subject\\/34964410\\/","playable":true,"cover":"https://img3.doubanio.com\\/view\\/photo\\/s_ratio_poster\\/public\\/p

In [51]:
html=json.loads(res)
html['subjects']

[{'rate': '6.4',
  'cover_x': 1187,
  'title': '白头山',
  'url': 'https://movie.douban.com/subject/30200087/',
  'playable': False,
  'cover': 'https://img1.doubanio.com/view/photo/s_ratio_poster/public/p2575684177.jpg',
  'id': '30200087',
  'cover_y': 1701,
  'is_new': False},
 {'rate': '7.7',
  'cover_x': 2416,
  'title': '决战中途岛',
  'url': 'https://movie.douban.com/subject/26786669/',
  'playable': False,
  'cover': 'https://img3.doubanio.com/view/photo/s_ratio_poster/public/p2573582192.jpg',
  'id': '26786669',
  'cover_y': 3375,
  'is_new': False},
 {'rate': '7.7',
  'cover_x': 1500,
  'title': '恶人传',
  'url': 'https://movie.douban.com/subject/30211551/',
  'playable': False,
  'cover': 'https://img3.doubanio.com/view/photo/s_ratio_poster/public/p2555084871.jpg',
  'id': '30211551',
  'cover_y': 2145,
  'is_new': False},
 {'rate': '5.5',
  'cover_x': 2580,
  'title': '吹哨人',
  'url': 'https://movie.douban.com/subject/30198715/',
  'playable': False,
  'cover': 'https://img3.doubanio.

## 多页

In [48]:
data=[]
for i in range(3):
    url="https://movie.douban.com/j/search_subjects?type=movie&tag=%E7%83%AD%E9%97%A8&sort=recommend&page_limit=20&page_start={}".format(i*20)
    res=requests.get(url,headers=header).text
    html=json.loads(res)['subjects']
    data+=html

In [50]:
movies_all=pd.DataFrame(data)
movies_all.head(1)

,cover,cover_x,cover_y,id,is_new,playable,rate,title,url
0,https://img3.doubanio.com/view/photo/s_ratio_p...,1080,1542,30306570,False,True,5.9,囧妈,https://movie.douban.com/subject/30306570/


In [53]:
movies=movies_all.loc[:,['title','rate','is_new','cover','url']]
movies.head(3)

,title,rate,is_new,cover,url
0,囧妈,5.9,False,https://img3.doubanio.com/view/photo/s_ratio_p...,https://movie.douban.com/subject/30306570/
1,1917,8.5,False,https://img1.doubanio.com/view/photo/s_ratio_p...,https://movie.douban.com/subject/30252495/
2,丑女之战,6.1,True,https://img3.doubanio.com/view/photo/s_ratio_p...,https://movie.douban.com/subject/30162754/


In [61]:
movies_new=movies.sort_values(by='rate',ascending=False)
movies_data=movies_new.reset_index(drop=True)   #删除排序后错乱的index索引
movies_data.head(3)

,title,rate,is_new,cover,url
0,爱尔兰人,8.9,False,https://img9.doubanio.com/view/photo/s_ratio_p...,https://movie.douban.com/subject/6981153/
1,多哥,8.8,False,https://img9.doubanio.com/view/photo/s_ratio_p...,https://movie.douban.com/subject/30271841/
2,小丑,8.7,False,https://img9.doubanio.com/view/photo/s_ratio_p...,https://movie.douban.com/subject/27119724/


## 保存在excel中

In [62]:
movies_data.to_excel('movies.xlsx',sheet_name='movies')